## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Georgetown,5.4112,100.3354,moderate rain,77.00,88,40,13.80,MY
1,1,Bredasdorp,-34.5322,20.0403,clear sky,62.08,92,1,1.88,ZA
2,2,Port Alfred,-33.5906,26.8910,broken clouds,67.60,82,58,19.44,ZA
3,3,Kimbe,-5.5502,150.1429,overcast clouds,83.95,70,99,1.23,PG
4,4,East London,-33.0153,27.9116,overcast clouds,67.62,80,98,11.30,ZA


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
new_df

,City_ID,City,Lat,Lng,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Georgetown,5.4112,100.3354,moderate rain,77.00,88,40,13.80,MY
1,1,Bredasdorp,-34.5322,20.0403,clear sky,62.08,92,1,1.88,ZA
2,2,Port Alfred,-33.5906,26.8910,broken clouds,67.60,82,58,19.44,ZA
4,4,East London,-33.0153,27.9116,overcast clouds,67.62,80,98,11.30,ZA
10,10,Lahat,-3.8000,103.5333,overcast clouds,71.20,97,99,2.42,ID
...,...,...,...,...,...,...,...,...,...,...
696,696,Camacha,33.0833,-16.3333,few clouds,68.00,83,20,2.30,PT
698,698,Navirai,-23.0650,-54.1906,clear sky,70.36,44,4,6.31,BR
701,701,Ballina,-28.8667,153.5667,clear sky,61.00,48,0,9.22,AU
704,704,Wilmington Island,32.0036,-80.9737,clear sky,60.80,36,1,9.22,US


In [17]:
# 4a. Determine if there are any empty rows.
new_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Weather Description    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                8
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_df = new_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Georgetown,MY,77.00,moderate rain,5.4112,100.3354,
1,Bredasdorp,ZA,62.08,clear sky,-34.5322,20.0403,
2,Port Alfred,ZA,67.60,broken clouds,-33.5906,26.8910,
4,East London,ZA,67.62,overcast clouds,-33.0153,27.9116,
10,Lahat,ID,71.20,overcast clouds,-3.8000,103.5333,
11,Rikitea,PF,77.90,light rain,-23.1203,-134.9692,
13,Christchurch,NZ,69.01,overcast clouds,-43.5333,172.6333,
16,Cayenne,GF,79.00,overcast clouds,4.9333,-52.3333,
19,Juba,SS,74.68,broken clouds,4.8517,31.5825,
25,Houma,US,68.00,clear sky,29.5958,-90.7195,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Georgetown,MY,77.00,moderate rain,5.4112,100.3354,Cititel Penang
1,Bredasdorp,ZA,62.08,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
2,Port Alfred,ZA,67.60,broken clouds,-33.5906,26.8910,The Halyards Hotel
4,East London,ZA,67.62,overcast clouds,-33.0153,27.9116,Tu Casa
10,Lahat,ID,71.20,overcast clouds,-3.8000,103.5333,Hotel Grand Zuri Lahat by ZHM
...,...,...,...,...,...,...,...
696,Camacha,PT,68.00,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
698,Navirai,BR,70.36,clear sky,-23.0650,-54.1906,Dubai Hotel - Também sua casa!
701,Ballina,AU,61.00,clear sky,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
704,Wilmington Island,US,60.80,clear sky,32.0036,-80.9737,Breezeway Studio Manor LLC


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
